In [ ]:
import pandas as pd
import requests
import zipfile
import io
import os
import csv
import ctypes as ct
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
import re
import seaborn as sns
import QuantLib as ql

from bgs.bgs_utils import clean_date, clean_percentage
from bgs.load_bgs_amounts import load_bgs_amounts
from bgs.load_gilt_details import load_csv_blocks
from bgs.load_bgs_prices import load_prices
from bgs.gilt_analytics import gilt_yield, yield_series
from bgs.linker_analytics import linker_real_yield

### British Gilt Survey Amounts Table

In [ ]:
tables = load_bgs_amounts("downloads/BGSAmounts.csv")

conv = tables['Conventionals']
conv.replace("Redeemed", "", inplace=True)
conv = conv.apply(pd.to_numeric, errors='coerce')
conv.fillna(0, inplace=True)
conv.index = pd.Index([clean_date(x) for x in list(conv.index)]).to_period('M').to_timestamp('M')
conv.columns = [np.int64(x.strip()) for x in conv.columns]

old = tables['Calculated indexed nominal Old-style']
old.replace("Redeemed", "", inplace=True)
old = old.apply(pd.to_numeric, errors='coerce')
old.fillna(0, inplace=True)
old.index = pd.Index([clean_date(x) for x in list(old.index)]).to_period('M').to_timestamp('M')
old.columns = [np.int64(x.strip()) for x in old.columns]

new = tables['Calculated indexed nominal New-style']
new.replace("Redeemed", "", inplace=True)
new = new.apply(pd.to_numeric, errors='coerce')
new.fillna(0, inplace=True)
new.index = pd.Index([clean_date(x) for x in list(new.index)]).to_period('M').to_timestamp('M')
new.columns = [np.int64(x.strip()) for x in new.columns]

new_no_idx = tables['Index-linked New-style']
new_no_idx.replace("Redeemed", "", inplace=True)
new_no_idx = new_no_idx.apply(pd.to_numeric, errors='coerce')
new_no_idx.fillna(0, inplace=True)
new_no_idx.index = pd.Index([clean_date(x) for x in list(new_no_idx.index)]).to_period('M').to_timestamp('M')
new_no_idx.columns = [np.int64(x.strip()) for x in new_no_idx.columns]

### British Gilts Survey Month End Prices

In [ ]:
price_df=load_prices("downloads/BGSPrices.csv")
price_df.index = pd.to_datetime(price_df.index, format="%d %b %Y").to_period('M').to_timestamp('M')
for text in ['Amalgamated', 'Redeemed', 'redeemed']:
    price_df = price_df.replace(text, 0)
price_df = price_df.replace('missing', None)
price_df = price_df.ffill(axis=0)
price_df = price_df.fillna(0)
price_df.columns = [np.int64(x) for x in price_df.columns]

### British Gilts Survey Details Table (Bond Static Data, Coupon Maturity etc.) 

In [ ]:
details = load_csv_blocks("downloads/BGSDetails.csv")

conv_details = details['Conventionals']
conv_details['%'] = conv_details['%'].apply(clean_percentage)
conv_details['Sequence'] = conv_details['Sequence'].apply(lambda x: np.int64(x))

new_details = details['Index-Linked New-style']
new_details['%'] = new_details['%'].apply(clean_percentage)
new_details['Sequence'] = new_details['Sequence'].apply(lambda x: np.int64(x))

old_details = details['Index-Linked Old-style']
old_details['%'] = old_details['%'].apply(clean_percentage)
old_details['Sequence'] = old_details['Sequence'].apply(lambda x: np.int64(x))

strip_details = details['Strips']
strip_details['%'] = 0.0
strip_details['Sequence'] = strip_details['Sequence'].apply(lambda x: np.int64(x))

date_variables = ["Latest redemption date","Issue date","First coupon payable on date",]
for date_var in date_variables:
    conv_details[date_var] = conv_details[date_var].apply(clean_date)
    old_details[date_var] = old_details[date_var].apply(clean_date)
    new_details[date_var] = new_details[date_var].apply(clean_date)
strip_details['Latest redemption date'] = strip_details['Latest redemption date'].apply(clean_date)

### Boe Interpolated Yield Curves

In [ ]:
boe_curves = "downloads/GLC Nominal month end data_2025 to present.xlsx"
df_curves = pd.ExcelFile(boe_curves)
df_curves.sheet_names

In [ ]:
df_curve = pd.read_excel(boe_curves, sheet_name='4. spot curve', skiprows=3)

### Fix a date

In [ ]:
calculation_date = "2025-01-31"

In [ ]:
def bgs_gilt_yield(bgs_id, calc_date):

    bond = conv_details[conv_details['Sequence']==bgs_id]
    issue_date=bond['Issue date'].dt.strftime("%Y-%m-%d").values[0]
    maturity_date=bond['Latest redemption date'].dt.strftime("%Y-%m-%d").values[0]
    first_cpn_date=bond['First coupon payable on date'].dt.strftime("%Y-%m-%d").values[0]
    last_cpn_date=bond['Latest redemption date'].dt.strftime("%Y-%m-%d").values[0]
    coupon=bond['%'].values[0]/100

    clean_price=price_df.loc[calc_date,bgs_id]


    return gilt_yield(
        trade_date=calc_date,
        issue_date=issue_date,
        maturity_date=maturity_date,
        first_cpn_date=first_cpn_date,
        last_cpn_date=last_cpn_date,
        clean_price=clean_price,
        coupon=coupon)
    

In [ ]:
bgs_gilt_yield(bgs_id=32282, calc_date=calculation_date)

In [ ]:
conv_details.columns

In [ ]:
all_conv = conv_details[['Sequence', 'ISIN Code','%','Latest redemption date', 'Issue date', 'First coupon payable on date']]
gilts_list = all_conv.loc[(all_conv['Latest redemption date'] > pd.to_datetime(calculation_date))&(all_conv['Issue date']< pd.to_datetime(calculation_date))]

In [ ]:
gilts_list['yield'] = gilts_list['Sequence'].apply(lambda x: bgs_gilt_yield(bgs_id=x, calc_date=calculation_date))

In [ ]:
df_curve.head()
df_curve.replace("2025-05-30",'2025-05-31', inplace=True)

In [ ]:
df_curve['years:'] = pd.to_datetime(df_curve['years:'], format="%Y-%m-%d")
df_curve.set_index('years:', inplace=True)

In [ ]:
df_curve.columns = [pd.to_datetime(x) for x in [pd.to_datetime(calculation_date)+ pd.DateOffset(months=12*x) for x in df_curve.columns]]

In [ ]:
spot_curve = df_curve.loc[calculation_date]

In [ ]:
spot_curve.index

In [ ]:
sns.set_theme(style="darkgrid")

# use the scatterplot function to build the bubble map
ax = sns.scatterplot(
    data=gilts_list,
    x="Latest redemption date",
    y="yield",
)
ax.plot(spot_curve.index, spot_curve, color='red')
plt.show()

In [ ]:
calculation_date = "2025-04-30"

spot_curve = df_curve.loc[pd.to_datetime(calculation_date)]
gilts_list = all_conv.loc[(all_conv['Latest redemption date'] > pd.to_datetime(calculation_date))&(all_conv['Issue date']< pd.to_datetime(calculation_date))]
gilts_list['yield'] = gilts_list['Sequence'].apply(lambda x: bgs_gilt_yield(bgs_id=x, calc_date=calculation_date))
sns.set_theme(style="darkgrid")

# use the scatterplot function to build the bubble map
ax = sns.scatterplot(
    data=gilts_list,
    x="Latest redemption date",
    y="yield",
)
ax.plot(spot_curve.index, spot_curve, color='red')
plt.show()

In [ ]:
df_curve.index

In [ ]:
len(ax.get_lines()[0].get_ydata())

In [ ]:
len(ax.get_lines()[0].get_xdata())

In [ ]:
gilts_list.shape

In [ ]:
gilts_list['low_coupon'] = gilts_list.apply(lambda row: 1 if row['%'] < 2.0 else 0, axis=1)

In [ ]:
gilts_list.columns

In [ ]:
sns.set_theme(style="darkgrid")

# use the scatterplot function to build the bubble map
ax = sns.scatterplot(
    data=gilts_list,
    x="Latest redemption date",
    y="yield",
    hue="low_coupon",
)
ax.plot(spot_curve.index, spot_curve, color='red')
plt.show()

In [ ]:
gilts_list.columns

In [ ]:
def bgs_running_yield(bgs_id, calc_date):

    bond = conv_details[conv_details['Sequence']==bgs_id]
    coupon=bond['%'].values[0]

    clean_price=price_df.loc[calc_date,bgs_id]


    return coupon/clean_price

In [ ]:
gilts_list['running_yield'] = gilts_list['Sequence'].apply(lambda x: bgs_running_yield(bgs_id=x, calc_date=calculation_date))   

In [ ]:
sns.set_theme(style="darkgrid")

# use the scatterplot function to build the bubble map
ax = sns.scatterplot(
    data=gilts_list,
    x="Latest redemption date",
    y="running_yield",
    hue="low_coupon",
)

In [ ]:
gilts_list[gilts_list['running_yield'] < 0.01]

In [ ]:
price_df.loc[calculation_date,32060]

In [ ]:
conv_details[conv_details['Sequence']==32060]

In [ ]:
gilts_list['returns_to_coupon'] = gilts_list['running_yield'] *100 /gilts_list['yield']

In [ ]:
sns.set_theme(style="darkgrid")

# use the scatterplot function to build the bubble map
ax = sns.scatterplot(
    data=gilts_list,
    x="Latest redemption date",
    y="returns_to_coupon",
    hue="low_coupon",
)

In [ ]:
gilts_list.columns

In [ ]:
gilts_list['price'] = gilts_list['Sequence'].apply(lambda x: price_df[x].loc[calculation_date])

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%load_ext autoreload
%autoreload 2
from bgs.gilt_analytics import yield_curves, from_iso, yield_curves_pw, yield_curve_rv

In [ ]:
c = yield_curves(calculation_date, gilts_list)

In [ ]:
c.keys()

In [ ]:
c["Svensson"].maxDate()

In [ ]:
gilts_list["yield"] = gilts_list["yield"]/100

In [ ]:
from matplotlib.dates import YearLocator
from matplotlib.ticker import PercentFormatter

f = plt.figure(figsize=(8, 5))
ax = f.add_subplot(1, 1, 1)

ax.xaxis.grid(True, "major", color="lightgray")
ax.yaxis.grid(True, "major", color="lightgray")
ax.xaxis.set_major_locator(YearLocator(10))

ax.yaxis.set_major_formatter(PercentFormatter(1.0))
styles = iter(["-", "--", ":", "-."])

dates = [from_iso(calculation_date) + ql.Period(i, ql.Months) for i in range(12 * 48 + 1)]

day_count = ql.ActualActual(ql.ActualActual.Bond)

for tag in c:
    rates = [
        c[tag].zeroRate(d, day_count, ql.Continuous).rate() for d in dates
    ]
    ax.plot_date(
        [d.to_date() for d in dates],
        rates,
        next(styles),
        label=tag,
    )
ax.legend(loc="best")

sns.scatterplot(
    data=gilts_list,
    x="Latest redemption date",
    y="yield",
    hue="low_coupon",
    ax=ax
)

In [ ]:
f = plt.figure(figsize=(8, 5))
ax = f.add_subplot(1, 1, 1)

ax.xaxis.grid(True, "major", color="lightgray")
ax.yaxis.grid(True, "major", color="lightgray")
ax.xaxis.set_major_locator(YearLocator(10))

ax.yaxis.set_major_formatter(PercentFormatter(1.0))
styles = iter(["-", "--", ":", "-."])

dates = [from_iso(calculation_date) + ql.Period(i, ql.Months) for i in range(12 * 48 + 1)]

day_count = ql.ActualActual(ql.ActualActual.Bond)

for tag in ['Nelson/Siegel', 'Exp. splines', 'Svensson']:
    rates = [
        c[tag].zeroRate(d, day_count, ql.Continuous).rate() for d in dates
    ]
    ax.plot_date(
        [d.to_date() for d in dates],
        rates,
        next(styles),
        label=tag,
    )
ax.legend(loc="best")

sns.scatterplot(
    data=gilts_list,
    x="Latest redemption date",
    y="yield",
    hue="low_coupon",
    ax=ax
)

In [ ]:
f = plt.figure(figsize=(8, 5))
ax = f.add_subplot(1, 1, 1)

ax.xaxis.grid(True, "major", color="lightgray")
ax.yaxis.grid(True, "major", color="lightgray")
ax.xaxis.set_major_locator(YearLocator(10))

ax.yaxis.set_major_formatter(PercentFormatter(1.0))
styles = iter(["-", "--", ":", "-."])

dates = [from_iso(calculation_date) + ql.Period(i, ql.Months) for i in range(12 * 48 + 1)]

day_count = ql.ActualActual(ql.ActualActual.Bond)

for tag in ['Nelson/Siegel', 'Svensson']:
    rates = [
        c[tag].zeroRate(d, day_count, ql.Continuous).rate() for d in dates
    ]
    ax.plot_date(
        [d.to_date() for d in dates],
        rates,
        next(styles),
        label=tag,
    )
ax.legend(loc="best")

sns.scatterplot(
    data=gilts_list,
    x="Latest redemption date",
    y="yield",
    hue="low_coupon",
    ax=ax
)

ax.plot(spot_curve.index, spot_curve/100, color='red')

In [ ]:
spot_curve

In [ ]:
high_coupon = gilts_list[gilts_list['low_coupon']==0]
high_coupon_curve = yield_curves(calculation_date, high_coupon)

In [ ]:
f = plt.figure(figsize=(8, 5))
ax = f.add_subplot(1, 1, 1)

ax.xaxis.grid(True, "major", color="lightgray")
ax.yaxis.grid(True, "major", color="lightgray")
ax.xaxis.set_major_locator(YearLocator(10))

ax.yaxis.set_major_formatter(PercentFormatter(1.0))
styles = iter(["-", "--", ":", "-."])

dates = [from_iso(calculation_date) + ql.Period(i, ql.Months) for i in range(12 * 43 + 1)]

day_count = ql.ActualActual(ql.ActualActual.Bond)

for tag in ['Nelson/Siegel', 'Svensson']:
    rates = [
        high_coupon_curve[tag].zeroRate(d, day_count, ql.Continuous).rate() for d in dates
    ]
    ax.plot_date(
        [d.to_date() for d in dates],
        rates,
        next(styles),
        label=tag,
    )
ax.legend(loc="best")

sns.scatterplot(
    data=gilts_list,
    x="Latest redemption date",
    y="yield",
    hue="low_coupon",
    ax=ax
)

ax.plot(spot_curve.index, spot_curve/100, color='red')

In [ ]:
pw = yield_curves_pw(calculation_date, high_coupon)

In [ ]:
pw.keys()

In [ ]:
key = 'splineCubicDiscount'

In [ ]:
f = plt.figure(figsize=(8, 5))
ax = f.add_subplot(1, 1, 1)

ax.xaxis.grid(True, "major", color="lightgray")
ax.yaxis.grid(True, "major", color="lightgray")
ax.xaxis.set_major_locator(YearLocator(10))

ax.yaxis.set_major_formatter(PercentFormatter(1.0))
styles = iter(["-", "--", ":", "-."])

dates = [from_iso(calculation_date) + ql.Period(i, ql.Months) for i in range(12 * 43 + 1)]

day_count = ql.ActualActual(ql.ActualActual.Bond)

for tag in ['Nelson/Siegel', 'Svensson']:
    rates = [
        high_coupon_curve[tag].zeroRate(d, day_count, ql.Continuous).rate() for d in dates
    ]
    ax.plot_date(
        [d.to_date() for d in dates],
        rates,
        next(styles),
        label=tag,
    )
ax.legend(loc="best")

sns.scatterplot(
    data=gilts_list,
    x="Latest redemption date",
    y="yield",
    hue="low_coupon",
    ax=ax
)

ax.plot(spot_curve.index, spot_curve/100, color='red')

rates_pw = [
        pw[key].zeroRate(d, day_count, ql.Continuous).rate() for d in dates
    ]
ax.plot_date(
        [d.to_date() for d in dates],
        rates_pw,
        next(styles),
        label=tag,
    )

In [ ]:
low_coupon = gilts_list[gilts_list['low_coupon']==1]
low_coupon_curve = yield_curves(calculation_date, low_coupon)

In [ ]:
f = plt.figure(figsize=(8, 5))
ax = f.add_subplot(1, 1, 1)

ax.xaxis.grid(True, "major", color="lightgray")
ax.yaxis.grid(True, "major", color="lightgray")
ax.xaxis.set_major_locator(YearLocator(10))
ax.set_ylim(0.02,0.06)

ax.yaxis.set_major_formatter(PercentFormatter(1.0))
styles = iter(["-", "--", ":", "-."])

dates = [from_iso(calculation_date) + ql.Period(i, ql.Months) for i in range(12 * 43 + 1)]

day_count = ql.ActualActual(ql.ActualActual.Bond)

for tag in ['Nelson/Siegel', 'Svensson']:
    rates = [
        low_coupon_curve[tag].zeroRate(d, day_count, ql.Continuous).rate() for d in dates
    ]
    ax.plot_date(
        [d.to_date() for d in dates],
        rates,
        next(styles),
        label=tag,
    )
ax.legend(loc="best")

sns.scatterplot(
    data=gilts_list,
    x="Latest redemption date",
    y="yield",
    hue="low_coupon",
    ax=ax
)

ax.plot(spot_curve.index, spot_curve/100, color='red')

In [ ]:
gilts_list.loc[gilts_list['yield']==gilts_list['yield'].min()]

In [ ]:
rv_df = yield_curve_rv(calculation_date, gilts_list)

In [ ]:
rv_df.columns

In [ ]:
(rv_df['yield'] - rv_df['curve_yield']).mean()

In [ ]:
(rv_df['price'] - rv_df['curve_price']).mean()